<a href="https://colab.research.google.com/github/balimpuramvikasgoud/FinVeda/blob/main/4_1assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving plant doc field.v1i.yolov5pytorch.zip to plant doc field.v1i.yolov5pytorch.zip


In [ ]:
import zipfile
import os

zip_path = "plant doc field.v1i.yolov5pytorch.zip"
extract_path = "/content/plant_dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Unzipped successfully!")


✅ Unzipped successfully!


In [ ]:
!pip install opencv-python


In [ ]:
import os
import cv2
import yaml

# Load class names from data.yaml
with open('/content/plant_dataset/data.yaml', 'r') as f:
    data_yaml = yaml.safe_load(f)
class_names = data_yaml['names']

def crop_objects(images_dir, labels_dir, output_dir='/content/cropped_dataset'):
    os.makedirs(output_dir, exist_ok=True)

    image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.jpg') or f.endswith('.png')])

    for img_file in image_files:
        img_path = os.path.join(images_dir, img_file)
        label_path = os.path.join(labels_dir, os.path.splitext(img_file)[0] + '.txt')

        if not os.path.exists(label_path):
            continue

        img = cv2.imread(img_path)
        h, w, _ = img.shape

        with open(label_path, 'r') as f:
            lines = f.readlines()

        for i, line in enumerate(lines):
            parts = line.strip().split()
            if len(parts) != 5:
                continue

            cls_id = int(parts[0])
            if cls_id >= len(class_names):  # ✅ Fix: check valid class ID
                print(f"⚠️ Skipping invalid class ID {cls_id} in file {label_path}")
                continue

            x_center, y_center, bw, bh = map(float, parts[1:])
            x1 = int((x_center - bw / 2) * w)
            y1 = int((y_center - bh / 2) * h)
            x2 = int((x_center + bw / 2) * w)
            y2 = int((y_center + bh / 2) * h)

            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(w, x2)
            y2 = min(h, y2)

            cropped = img[y1:y2, x1:x2]
            if cropped.size == 0:
                continue

            cname = class_names[cls_id]
            class_dir = os.path.join(output_dir, cname)
            os.makedirs(class_dir, exist_ok=True)

            save_path = os.path.join(class_dir, f"{os.path.splitext(img_file)[0]}_{i}.jpg")
            cv2.imwrite(save_path, cropped)
#Removes background from images.
#
#Focuses on only the leaf/disease part for better classification accuracy.

In [ ]:
from torchvision import transforms

transform = transforms.Compose([                #
    transforms.Resize((224, 224)),             # Resize to model-compatible size
    transforms.ToTensor(),                     # Convert to tensor
    transforms.Normalize([0.5, 0.5, 0.5],       # Normalize 3-channel images
                         [0.5, 0.5, 0.5])
])


#Data pipelines are like a factory line for your data — every raw image enters, goes through the same cleaning and shaping process, and comes out in a format the model can directly use.


In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import os

class CroppedPlantDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_to_idx = {}

        # First, get all subdirectories that are actual class folders
        class_dirs = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(sorted(class_dirs))}

        for cls_name in class_dirs:
            cls_folder = os.path.join(root_dir, cls_name)
            for img_name in os.listdir(cls_folder):
                img_path = os.path.join(cls_folder, img_name)
                if os.path.isfile(img_path):
                    self.image_paths.append(img_path)
                    self.labels.append(self.class_to_idx[cls_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

        #In simple words, this code creates a custom PyTorch dataset that loads your cropped plant images and their labels so the model can train on them.


In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Replace the path with your cropped image dataset folder
dataset = CroppedPlantDataset("/content/cropped_dataset", transform=transform)
#This code prepares your cropped plant images so they are ready for training.
#"Convert images into tensors" means changing the image from a normal picture format (like JPEG/PNG) into a numerical array that PyTorch can understand and use for training.
#Resize all images to 224×224 pixels (same size).




In [ ]:
from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
#This code splits your dataset into two parts:

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
#This code prepares your training and validation data so the model can read it in small groups (batches) instead of all at once.


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)

# Modify the final layer to match your number of classes
num_classes = len(dataset.class_to_idx)
model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 88.6MB/s]


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
#Loss function = “How wrong am I?”

#Optimizer = “How should I change to be less wrong?”


In [ ]:
from torch.utils.data import random_split, DataLoader

# 80% training, 20% validation split
total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size = total_size - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
#This code splits your dataset into two parts and prepares them for training.


In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")
#This code is training your model for 10 rounds (epochs) and showing how well it’s learning after each round.

Epoch [1/10], Loss: 1.3263, Accuracy: 43.48%
Epoch [2/10], Loss: 0.1133, Accuracy: 95.65%
Epoch [3/10], Loss: 0.0344, Accuracy: 100.00%
Epoch [4/10], Loss: 0.0034, Accuracy: 100.00%
Epoch [5/10], Loss: 0.0013, Accuracy: 100.00%
Epoch [6/10], Loss: 0.0007, Accuracy: 100.00%
Epoch [7/10], Loss: 0.0004, Accuracy: 100.00%
Epoch [8/10], Loss: 0.0003, Accuracy: 100.00%
Epoch [9/10], Loss: 0.0002, Accuracy: 100.00%
Epoch [10/10], Loss: 0.0002, Accuracy: 100.00%


In [ ]:
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, target_names=dataset.class_to_idx.keys()))
#This code is checking how well the trained model works on the validation data and then printing a detailed performance report.




                   precision    recall  f1-score   support

                1       0.00      0.00      0.00         2
        CC BY 4.0       1.00      0.50      0.67         2
  plant-doc-field       0.00      0.00      0.00         0
tru-project-qvb6r       0.50      1.00      0.67         2

         accuracy                           0.50         6
        macro avg       0.38      0.38      0.33         6
     weighted avg       0.50      0.50      0.44         6



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving cornleaf.jpg to cornleaf.jpg


In [ ]:
import os

# Get the uploaded image filename
image_filename = next(iter(uploaded))
print(f"Uploaded file: {image_filename}")


Uploaded file: cornleaf.jpg


In [ ]:
from PIL import Image
import torchvision.transforms as transforms
import torch

# Define same transforms as training
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load and preprocess image
img = Image.open(image_filename).convert("RGB")
img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

# Ensure model is in evaluation mode
model.eval()

# Make prediction
with torch.no_grad():
    outputs = model(img_tensor)
    _, predicted = torch.max(outputs, 1)

# Map prediction to class name
predicted_class = class_names[predicted.item()]
print("Predicted class:", predicted_class)


Predicted class: corn-leaf-rust
